Next Step:
- Download the data from [here](https://www.govinfo.gov/bulkdata/CFR)

- ⬇️ Downloads XML data files per agency reference (full titles or chapters), and stores raw regulation text

In [ ]:
# Print config for verification
#print(f"Snapshot Date: {snapshot_date}")

########### for 02 notebook.......
# Access paths and config
# regulation_text_path = paths["REGULATION_TEXT_PATH"]  # i.e. ~/repo/doge-data-challenge/regulation_text/{SNAPSHOT_DATE}

################# for 02 or 03 or 04 notebooks.......
# Print config for verification
# print(f"Regulation Text Path: {shorten_path(regulation_text_path)}")
# print(f"Word Count Method: {config['WORDCOUNT_METHOD']}")


In [67]:
import sys
sys.path.append("..")  # So notebooks/utils is importable from inside notebooks/

from utils.print_helpers import shorten_path
from pathlib import Path

# Snapshot date (can be reused across all notebooks)
SNAPSHOT_DATE = "2025-03-27"

# Go up one level from the notebook directory to reach project root
PROJECT_ROOT = Path.cwd().resolve().parent

# Key directories relative to project root
ARCHIVE_DIR = PROJECT_ROOT / "archive"
XML_DIR = PROJECT_ROOT / "data" / "regulations_xml" / SNAPSHOT_DATE
CSV_PATH = ARCHIVE_DIR / f"flattened_agencies_list_{SNAPSHOT_DATE}.csv"


# Build relative directories
#DATA_DIR = PROJECT_ROOT / "data" / "regulations_xml" / snapshot_date
#CSV_PATH = PROJECT_ROOT / "archive" / f"flattened_agencies_list_{snapshot_date}.csv"

#########################################################
# Load the flattened agency list generated in notebook 01
#########################################################
# csv_path = "../archive/flattened_agencies_list_2025-04-06.csv"

print(shorten_path(ARCHIVE_DIR))
print(shorten_path(XML_DIR))
print(shorten_path(CSV_PATH))

# should the snapshot date be incorporated into the flattened_agencies_list_<date>.CSV file?? I think that makes sense
# define important top level variables

~/repo/doge-data-challenge/archive
~/repo/doge-data-challenge/data/regulations_xml/2025-03-27
~/repo/doge-data-challenge/archive/flattened_agencies_list_2025-03-27.csv


In [68]:


import pandas as pd
import os

if os.path.exists(CSV_PATH):
    agencies_df = pd.read_csv(CSV_PATH)
    print(f"✅ Loaded {len(agencies_df)} rows from {shorten_path(CSV_PATH)}")
else:
    raise FileNotFoundError(f"❌ Could not find {shorten_path(CSV_PATH)}")

✅ Loaded 487 rows from ~/repo/doge-data-challenge/archive/flattened_agencies_list_2025-03-27.csv


In [ ]:
import requests
import time

xml_dir = os.path.join("data", "regulations_xml", snapshot_date)
os.makedirs(xml_dir, exist_ok=True)

titles = sorted(agencies_df["title"].dropna().unique())

print(f"\n🚀 Downloading {len(titles)} unique title XML files to: {xml_dir}\n")

total_start = time.time()

for title in titles:
    title = int(title)
    url = f"https://www.ecfr.gov/api/versioner/v1/full/{snapshot_date}/title-{title}.xml"
    filename = f"title_{title}.xml"
    save_path = os.path.join(xml_dir, filename)

    if os.path.exists(save_path):
        print(f"🟡 Skipping cached: {filename}")
        continue

    try:
        start = time.time()
        response = requests.get(url, timeout=(10, 60))
        elapsed = time.time() - start

        if response.status_code == 200:
            with open(save_path, "wb") as f:
                f.write(response.content)
            print(f"✅ Downloaded: {filename} ({elapsed:.2f}s)")
        else:
            print(f"⚠️ Failed to download {filename} (status {response.status_code})")

    except requests.RequestException as e:
        print(f"❌ Error downloading {filename} → {e}")

total_elapsed = time.time() - total_start
print(f"\n⏱️ Total elapsed time: {total_elapsed:.2f} seconds")